In [53]:
# Links for assistance on code:
# https://betterprogramming.pub/how-to-scrape-multiple-pages-of-a-website-using-a-python-web-scraper-4e2c641cff8
# https://stackoverflow.com/questions/53282445/scrape-a-table-from-multiple-pages-and-store-in-a-single-dataframe

from time import sleep
from random import randint
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib.request as urllib



In [90]:
# Scraping Headers off of the table so we can use in our data frame
site = urllib.urlopen('https://www.capfriendly.com/browse/active?stats-season=2021&display=skater-individual-advanced-stats,skater-on-ice-advanced-stats,goalie-advanced-stats&hide=handed')
soup = BeautifulSoup(site.read())
head = []
headers = soup.find_all('th')
for val in headers:
    head.append(str(val.get_text()))

['PLAYER',
 'TEAM',
 'AGE',
 'POS',
 'GP',
 'G',
 'A',
 'P',
 'P/GP',
 '+/-',
 'Sh',
 'Sh%',
 'TOI',
 'ixG',
 'iSh',
 'iCF',
 'iFF',
 'ixG60',
 'iSh60',
 'iCF60',
 'iFF60',
 'SF',
 'SA',
 'SF%',
 'CF',
 'CA',
 'CF%',
 'FF',
 'FA',
 'FF%',
 'xGF',
 'xGA',
 'xGF%',
 'W',
 'L',
 'SO',
 'GAA',
 'Sv%',
 'GA60',
 'xGA60',
 'GSAx60',
 'CLAUSE',
 'EXPIRY',
 'CAP HIT',
 'SALARY']

In [125]:
# could have made this scalable by writing a program, did not, whoops.

page = 1
data = []
# creating while loop to iterate through as many pages as we need, 31 is max, so stop at 32. 
while page != 32:
# first page does not follow the syntax of the rest of links, 
# so if list is empty add first page data to it.    
    if not data:
        newsite = urllib.urlopen('https://www.capfriendly.com/browse/active?stats-season=2021&display=skater-individual-advanced-stats,skater-on-ice-advanced-stats,goalie-advanced-stats&hide=handed')
        soups = BeautifulSoup(newsite.read())
        table = soups.find('tbody')
        rows = table.find_all('tr')
    # iterating through rows in tables and appending data in cells to list
        for row in rows:
            cells = row.find_all('td')
            data.append([cell.text for cell in cells])
        page += 1
    # adding sleep function so we dont get blocked from scraping.
        sleep(randint(2,10))
  
    else:
    # adding variable 'page' to end of url to find particular link we are looking for
        newsite = urllib.urlopen(f'https://www.capfriendly.com/browse/active?stats-season=2021&display=skater-individual-advanced-stats,skater-on-ice-advanced-stats,goalie-advanced-stats&hide=handed&pg={page}')
        soups = BeautifulSoup(newsite.read())
        table = soups.find('tbody')
        rows = table.find_all('tr')
        for row in rows:
            cells = row.find_all('td')
            data.append([cell.text for cell in cells])
        page += 1
        sleep(randint(2,10))
# converting list to dataframe, adding list head as column headers.
df = pd.DataFrame(data,columns=head).drop(0, axis=0)
df


,PLAYER,TEAM,AGE,POS,GP,G,A,P,P/GP,+/-,...,SO,GAA,Sv%,GA60,xGA60,GSAx60,CLAUSE,EXPIRY,CAP HIT,SALARY
1,2. Artemi Panarin,NYR,28,LW,42,17,41,58,1.38,14,...,-,-,-,-,-,-,NMC,UFA,"$11,642,857","$13,000,000"
2,3. Auston Matthews,TOR,22,C,52,41,25,66,1.27,21,...,-,-,-,-,-,-,,UFA,"$11,640,250","$15,900,000"
3,4. Erik Karlsson,SJS,30,RD,52,8,14,22,0.42,-18,...,-,-,-,-,-,-,NMC,UFA,"$11,500,000","$12,000,000"
4,5. Drew Doughty,LAK,30,RD,56,8,26,34,0.61,-14,...,-,-,-,-,-,-,NMC,UFA,"$11,000,000","$10,000,000"
5,6. John Tavares,TOR,29,C,56,19,31,50,0.89,12,...,-,-,-,-,-,-,NMC,UFA,"$11,000,000","$12,000,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,1506. Antti Suomela,TOR,26,C,4,0,0,0,0.00,0,...,-,-,-,-,-,-,,UFA,"$700,000","$700,000"
1506,1507. Michael Amadio,OTT,24,"C, RW",25,0,3,3,0.12,-3,...,-,-,-,-,-,-,,RFA,"$700,000","$700,000"
1507,1508. Gustav Forsling,FLA,24,LD,43,5,12,17,0.40,17,...,-,-,-,-,-,-,,RFA,"$700,000","$700,000"
1508,1509. David Gust,CAR,26,RW,0,0,0,0,0.00,0,...,-,-,-,-,-,-,,UFA,"$700,000","$700,000"


In [131]:
# Original data includes players name along with ranking of salary, have to split.
df[['rank','Player','2','3']] = df['PLAYER'].str.split('.',expand=True)
df = df.drop(columns = ['2','3','PLAYER'])
df

,TEAM,AGE,POS,GP,G,A,P,P/GP,+/-,Sh,...,Sv%,GA60,xGA60,GSAx60,CLAUSE,EXPIRY,CAP HIT,SALARY,rank,Player
1,NYR,28,LW,42,17,41,58,1.38,14,106,...,-,-,-,-,NMC,UFA,"$11,642,857","$13,000,000",2,Artemi Panarin
2,TOR,22,C,52,41,25,66,1.27,21,222,...,-,-,-,-,,UFA,"$11,640,250","$15,900,000",3,Auston Matthews
3,SJS,30,RD,52,8,14,22,0.42,-18,110,...,-,-,-,-,NMC,UFA,"$11,500,000","$12,000,000",4,Erik Karlsson
4,LAK,30,RD,56,8,26,34,0.61,-14,101,...,-,-,-,-,NMC,UFA,"$11,000,000","$10,000,000",5,Drew Doughty
5,TOR,29,C,56,19,31,50,0.89,12,163,...,-,-,-,-,NMC,UFA,"$11,000,000","$12,000,000",6,John Tavares
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,TOR,26,C,4,0,0,0,0.00,0,4,...,-,-,-,-,,UFA,"$700,000","$700,000",1506,Antti Suomela
1506,OTT,24,"C, RW",25,0,3,3,0.12,-3,24,...,-,-,-,-,,RFA,"$700,000","$700,000",1507,Michael Amadio
1507,FLA,24,LD,43,5,12,17,0.40,17,94,...,-,-,-,-,,RFA,"$700,000","$700,000",1508,Gustav Forsling
1508,CAR,26,RW,0,0,0,0,0.00,0,0,...,-,-,-,-,,UFA,"$700,000","$700,000",1509,David Gust


In [132]:
# when scraping for some reason omitted first row of first link, manually adding.
df.loc[len(df.index)] = ['EDM',23,'C',56,33,72,105,1.88,21,200,.17,'22:09',24.25,195,307,251,1.2,9.68,15.23,12.45,854,524,61.97,1508,912,62.31,1141,709,61.68,97.47,43.47,69.16,'-','-','-','-','-','-','-','-','','UFA','$12,500,000','$14,000,000',1,'Connor McDavid'] 

In [140]:
# rearraging columns and adding dataframe as csv to my library.
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]
df = df[cols]
df.to_csv('/Users/eliaskarikas/Desktop/DS3000/project/nhldata.csv')

In [141]:
cols

['CLAUSE',
 'EXPIRY',
 'CAP HIT',
 'SALARY',
 'rank',
 'Player',
 'TEAM',
 'AGE',
 'POS',
 'GP',
 'G',
 'A',
 'P',
 'P/GP',
 '+/-',
 'Sh',
 'Sh%',
 'TOI',
 'ixG',
 'iSh',
 'iCF',
 'iFF',
 'ixG60',
 'iSh60',
 'iCF60',
 'iFF60',
 'SF',
 'SA',
 'SF%',
 'CF',
 'CA',
 'CF%',
 'FF',
 'FA',
 'FF%',
 'xGF',
 'xGA',
 'xGF%',
 'W',
 'L',
 'SO',
 'GAA',
 'Sv%',
 'GA60',
 'xGA60',
 'GSAx60']